In [ ]:
import os
import numpy as np
from scipy.spatial.distance import euclidean
import xlrd
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import font as tkfont  # Import tkinter's font module

def calculate_distance(coord1, coord2):
    return euclidean(coord1, coord2)

def check_distance_range(df, distance_threshold):
    results = []
    for i in range(1, len(df)):
        point1 = (df.at[i - 1, 'X'], df.at[i - 1, 'Y']) #You can change the DF according to your excel.!
        point2 = (df.at[i, 'X'], df.at[i, 'Y'])

        distance = calculate_distance(point1, point2)
        rounded_distance = round(distance, 2)

        if rounded_distance > distance_threshold:
            result_text = f"'{df.at[i, 'POINT_DESCRIPTION']}' value is not in range, the distance is {rounded_distance} meters from '{df.at[i-1, 'POINT_DESCRIPTION']}'"
            results.append(result_text) #You can change the DF according to your excel.!

    return results

def process_files(input_file, output_folder, distance_threshold):
    df = pd.read_excel(input_file)
    df.sort_values(by='POINT_LOCATION', inplace=True) #You can change the DF according to your excel.!

    groups = df.groupby('POINT_LOCATION')
    
    output_text = []
    for _, group_df in groups:
        output_text.extend(check_distance_range(group_df, distance_threshold))

    output_file = os.path.join(output_folder, 'output.txt')
    with open(output_file, 'w') as f:
        f.write('\n'.join(output_text))

    print(f"Output file created at: {output_file}")
    messagebox.showinfo("Successful!", "Process completed successfully.")

class Application(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.title("Skipped Points Query")
        self.geometry("500x400")  # Set window size

        header_font = tkfont.Font(family="Helvetica", size=14, weight="bold")  # Use tkfont instead of tk.font

        tk.Label(self, text="Select Input .xls file:", font=header_font).pack()
        self.input_file = tk.Entry(self)
        self.input_file.pack()
        tk.Button(self, text="Browse", command=self.browse_input).pack()

        tk.Label(self, text="Select Output Folder:", font=header_font).pack()
        self.output_folder = tk.Entry(self)
        self.output_folder.pack()
        tk.Button(self, text="Browse", command=self.browse_output).pack()

        tk.Label(self, text="Enter Distance Threshold in Meters:", font=header_font).pack()
        self.distance_threshold = tk.Entry(self)
        self.distance_threshold.pack()

        tk.Label(self, text="").pack()  # Add an empty label for spacing

        run_button = tk.Button(self, text="Run", command=self.run_script, width=20, height=2, bg="green", fg="white")
        run_button.pack()

    def browse_input(self):
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xls")])
        self.input_file.delete(0, tk.END)
        self.input_file.insert(0, file_path)

    def browse_output(self):
        folder_path = filedialog.askdirectory()
        self.output_folder.delete(0, tk.END)
        self.output_folder.insert(0, folder_path)

    def run_script(self):
        input_file = self.input_file.get()
        output_folder = self.output_folder.get()

        try:
            distance_threshold = float(self.distance_threshold.get())
        except ValueError:
            messagebox.showerror("Error", "Please enter a valid numeric distance threshold.")
            return

        if not input_file or not output_folder:
            messagebox.showerror("Error", "Please select input file and output folder.")
            return

        process_files(input_file, output_folder, distance_threshold)

if __name__ == "__main__":
    app = Application()
    app.mainloop()
